# Gold: Création du modèle dimensionnel

### **Dimensions :**
1. **dim_date** : Informations calendaires
2. **dim_time** : Informations horaires
3. **dim_turbine** : Caractéristiques des turbines
4. **dim_operational_status** : Statuts opérationnels

### **Fait :**
- **fact_wind_power** : Mesures de productio

In [1]:
import pyspark.sql.functions as sf

df_silver = spark.read.load("abfss://79daad76-9b51-4f79-b1a3-5049702b5055@onelake.dfs.fabric.microsoft.com/f584c5bb-e8c1-42fe-ae55-db6de3ba417b/Tables/dbo/wind_power")

print(df_silver.dtypes)
display(df_silver.orderBy(sf.rand()).limit(5))

StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 3, Finished, Available, Finished, False)

[('production_id', 'bigint'), ('date', 'timestamp'), ('time', 'string'), ('turbine_name', 'string'), ('capacity', 'bigint'), ('location_name', 'string'), ('latitude', 'double'), ('longitude', 'double'), ('region', 'string'), ('status', 'string'), ('responsible_department', 'string'), ('wind_speed', 'double'), ('wind_direction', 'string'), ('energy_produced', 'double'), ('day', 'int'), ('month', 'int'), ('quarter', 'int'), ('year', 'int'), ('hour_of_day', 'string'), ('minute_of_hour', 'string'), ('second_of_minute', 'string'), ('time_period', 'string')]


SynapseWidget(Synapse.DataFrame, 51348e2d-5ad7-49d1-ab8a-89fddda4be48)

In [18]:
# dim_time
dim_time_df = df_silver.select(
    "time", 
    "hour_of_day", 
    "minute_of_hour", 
    "second_of_minute", 
    "time_period"
).distinct().withColumnRenamed("time", "time_id")

display(dim_time_df.orderBy(sf.rand()).limit(5))

StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 20, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, ea79bff3-b9b4-4f3a-8c31-7878b711f98b)

In [17]:
# dim_date
dim_date_df = df_silver.select(
    "date",
    "day",
    "month", 
    "quarter",
    "year"
).distinct().withColumnRenamed("date", "date_id")

display(dim_date_df.orderBy(sf.rand()).limit(5))

StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 19, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 51636f37-54fc-4999-9ed8-54addeaab9c5)

In [19]:
from pyspark.sql.window import Window 

# dim_turbine

dim_turbine_df = df_silver.select(
    "turbine_name", 
    "capacity",
    "location_name",
    "latitude",
    "longitude",
    "region"
).distinct()

dim_turbine_df = dim_turbine_df.withColumn(
    "turbine_id", 
    sf.row_number().over(Window.orderBy(
        "turbine_name", 
        "capacity", 
        "location_name"
    ))
)

display(dim_turbine_df.limit(5))

StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 21, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 2d4ed147-d309-4213-9017-f3bd4683c021)

In [20]:
# dim_operational_status
dim_operational_status_df = (df_silver
    .select(
        "status", 
        "responsible_department"
    ).distinct()
    .withColumn(
        "status_id", 
        sf.row_number().over(Window.orderBy(
            "status", 
            "responsible_department"
            )
        )
    )
)

display(dim_operational_status_df.limit(5))


StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 22, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 156efaa1-9478-4588-abe4-8418faf23a5f)

In [21]:
fact_wind_power_production_df = (df_silver
    .join(dim_operational_status_df, on=["status", "responsible_department"], how="left")
    .join(dim_turbine_df, on=["turbine_name", "capacity", "location_name"], how="left")
    .select(
        "production_id",
        "status_id",
        "turbine_id",
        sf.col("date").alias("date_id"),
        sf.col("time").alias("time_id"),
        "wind_speed",
        "wind_direction",
        "energy_produced"
    ).withColumnRenamed("date", "date_id")
    .withColumnRenamed("time", "time_id")
)

display(fact_wind_power_production_df.limit(5))

StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 23, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 9742b5ca-184c-443c-ad24-8587b471c49c)

In [23]:
tables = {
    "dim_date": dim_date_df,
    "dim_time": dim_time_df,
    "dim_turbine": dim_turbine_df,
    "dim_operational_status": dim_operational_status_df,
    "fact_wind_power": fact_wind_power_production_df
}

gold_base_path = "abfss://79daad76-9b51-4f79-b1a3-5049702b5055@onelake.dfs.fabric.microsoft.com/bbf29066-2e67-4f6c-9d58-f346cb97d758/Tables/dbo/"

for table, df in tables.items():
    df.write.format("delta").mode("overwrite").save(f"{gold_base_path}{table}")


StatementMeta(, a0fd3949-829e-486d-8507-319e54126146, 25, Finished, Available, Finished, False)